In [ ]:
import base
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns

# enables concurrent editing of base.py
from importlib import reload
reload(base)

sns.set_style('ticks')
sns.set_context('talk',rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})

### Setup

- Load data
- Add metadata
- Draw gates
- Gate transfected cells

Result from this section: DataFrame `df` representing transfected cells.

Load all transfection data collected as of 2024.04.11

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'/'kasey'

exp90_path = base_path/'2024.03.31_exp90'/'export'
exp90_2_path = base_path/'2024.04.02_exp90.2'/'export'
exp90_3_path = base_path/'2024.04.02_exp90.3'/'export'
exp90_4_path = base_path/'2024.04.05_exp90.4'/'export'
exp91_path = base_path/'2024.04.08_exp91'/'export'
exp92_path = base_path/'2024.04.12_exp92'/'export'

plates = pd.DataFrame({
    'data_path': [exp90_path/'plate1', exp90_path/'plate2', 
                  exp90_2_path, exp90_4_path,
                  exp90_3_path/'plate1', exp90_3_path/'plate2', 
                  exp91_path/'plate1.1', exp91_path/'plate1.2', exp91_path/'plate1.3', 
                  exp91_path/'plate2.1', exp91_path/'plate2.2', exp91_path/'plate2.3',
                  exp92_path/'plate1.1', exp92_path/'plate1.2', exp92_path/'plate1.3', 
                  exp92_path/'plate2.1', exp92_path/'plate2.2', exp92_path/'plate2.3',],
    
    'yaml_path': ([exp90_path/'exp90_plate1_wells.yaml', exp90_path/'exp90_plate2_wells.yaml', 
                   exp90_path/'exp90_plate2_wells.yaml', exp90_path/'exp90_plate1_wells.yaml',
                   exp90_path/'exp90_plate1_wells.yaml', exp90_path/'exp90_plate2_wells.yaml', ] +
                  [exp91_path/'exp91_plate1_wells.yaml']*3 + 
                  [exp91_path/'exp91_plate2.1_wells.yaml', exp91_path/'exp91_plate2.2_wells.yaml', exp91_path/'exp91_plate2.3_wells.yaml'] +
                  [exp92_path/'exp92_plate1_wells.yaml', exp92_path/'exp92_plate1.2_wells.yaml', exp92_path/'exp92_plate1_wells.yaml',
                   exp92_path/'exp92_plate2_wells.yaml', exp92_path/'exp92_plate2.2_wells.yaml', exp92_path/'exp92_plate2_wells.yaml',]
                  ),
    
    'biorep': ([1, 1, 
                2, 2, 
                3, 3,] + 
                [1, 2, 3,]*4),
    
    'exp': (['exp90', 'exp90', 
             'exp90.2', 'exp90.4', 
             'exp90.3', 'exp90.3',] + 
            ['exp91']*6 + 
            ['exp92']*6)
})

output_path = rd.rootdir/'output'/'tuning'/'more'
cache_path = rd.rootdir/'output'/'tuning'/'data.gzip'

metadata_keys = set()
for p in plates['yaml_path'].unique():
    rd.plot.plot_well_metadata(p)
    metadata_keys.update(rd.flow.load_well_metadata(p).keys())
display(metadata_keys)

In [ ]:
# Load data
data = pd.DataFrame()
if cache_path.is_file(): data = pd.read_parquet(cache_path)
else: 
    channel_list = ['mCherry-A','mRuby2-A','FSC-A','SSC-A','tagBFP-A','mGL-A']
    data = rd.flow.load_groups_with_metadata(plates, columns=channel_list)

    # Remove negative channel values
    for c in channel_list: data = data[data[c]>0]
    
    data.dropna(inplace=True)
    data.to_parquet(rd.outfile(cache_path))
display(data)

In [ ]:
# Add metadata for constructs
metadata = base.get_metadata(rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct-metadata.xlsx')
data = data.merge(metadata, how='left', on='construct')
display(data)

# Create dicts to specify colors/markers
metadata['miR_'] = (metadata['miR'].isin(['none','na'])).map({True: '–', False:'+'})
metadata['ts_'] = (metadata['ts'].isin(['none','na'])).map({True: '–', False:'+'})
metadata_dict = metadata.set_index('construct').to_dict('dict')
construct_palette = metadata_dict['color'] # colored by design
construct_markers = metadata_dict['markers']

In [ ]:
# Create unified color palette for most tuning
metadata2 = metadata.copy()
metadata2.loc[(metadata2['group']=='marker'), 'color'] = 'black'
metadata2.loc[(metadata2['group']=='base'), 'color'] = 'black'

metadata2.loc[(metadata2['group']=='miR'), 'color'] = base.colors['purple']
metadata2.loc[(metadata2['group']=='miR') & (metadata2['miR'].isin(['miR.FF5','miRE.FF5'])), 'color'] = metadata2.loc[(metadata2['group']=='miR') & (metadata2['miR'].isin(['miR.FF5','miRE.FF5'])), 'color'].apply(base.get_light_color)

metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])), 'color'] = base.colors['blue']
metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts'].isin(['FF6x1','FF6x2','FF6x4'])), 'color'] = metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts'].isin(['FF6x1','FF6x2','FF6x4'])), 'color'].apply(base.get_dark_color)
metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts'].isin(['FF4x1','FF4x2','FF4x4'])), 'color'] = metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts'].isin(['FF4x1','FF4x2','FF4x4'])), 'color'].apply(base.get_light_color)
metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts']=='FF3x1'), 'color'] = metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts']=='FF3x1'), 'color'].apply(base.get_light_color).apply(base.get_light_color)

metadata2.loc[(metadata2['group']=='controller') & (metadata2['miR']=='miR.FF5'), 'color'] = base.colors['red']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['miR']=='miRE.FF5'), 'color'] = base.colors['orange']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['miR']=='miR.FF4'), 'color'] = base.colors['green']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['miR']=='miRE.FF4'), 'color'] = base.colors['teal']

metadata2.loc[(metadata2['group']=='controller') & (metadata2['ts_kind']=='NT'), 'color'] = base.colors['gray']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['ts_kind']=='NT') & (metadata2['miR'].isin(['miR.FF4','miRE.FF4'])), 'color'] = metadata2.loc[(metadata2['group']=='controller') & (metadata2['ts_kind']=='NT') & (metadata2['miR'].isin(['miR.FF4','miRE.FF4'])), 'color'].apply(base.get_light_color)

metadata2.loc[((metadata2['group']=='controller') & (metadata2['ts_num']==2)), 'color'] = metadata2.loc[((metadata2['group']=='controller') & (metadata2['ts_num']==2)), 'color'].apply(base.get_light_color)
metadata2.loc[((metadata2['group']=='controller') & (metadata2['ts_num']==4)), 'color'] = metadata2.loc[((metadata2['group']=='controller') & (metadata2['ts_num']==4)), 'color'].apply(base.get_dark_color)
#metadata2.loc[((metadata2['ts_kind']=='NT') & (metadata2['ts_num']==2)), 'color'] = base.colors['gray']

metadata2.loc[(metadata2['ts']=='FF6x3.4x1'), 'color'] = metadata2.loc[(metadata2['ts']=='FF6x3.4x1'), 'color'].apply(base.get_light_color)

# markers
metadata2.loc[(metadata2['miR'].isin(['miR.FF5','miR.FF4'])), 'markers'] = 'D'
metadata2.loc[(metadata2['miR'].isin(['miRE.FF5','miRE.FF4'])), 'markers'] = 'o'
metadata2.loc[(metadata2['group']=='base'), 'markers'] = 'X'
metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])), 'markers'] = '^'
metadata2.loc[(metadata2['group']=='controller') & (metadata2['promoter']=='CAG'), 'markers'] = 'D'
metadata2.loc[(metadata2['group']=='controller') & (metadata2['promoter']=='EFS'), 'markers'] = '^'
metadata2.loc[(metadata2['group']=='controller') & (metadata2['promoter']=='hPGK'), 'markers'] = 'v'

# linestyles
metadata2['linestyle'] = '-' # default is solid
metadata2.loc[metadata2['group']=='marker', 'linestyle'] = ':'
metadata2.loc[(metadata2['miR'].isin(['miR.FF5','miR.FF4'])), 'linestyle'] = '--'

metadata_dict2 = metadata2.set_index('construct').to_dict('dict')
main_palette = metadata_dict2['color']
main_markers = metadata_dict2['markers']

In [ ]:
# Create color palette for comparing designs
metadata2 = metadata.copy()
metadata2.loc[(metadata2['group']=='marker'), 'color'] = 'black'
metadata2.loc[(metadata2['group']=='base'), 'color'] = 'black'

metadata2.loc[(metadata2['group']=='miR'), 'color'] = base.colors['purple']
metadata2.loc[(metadata2['group']=='miR') & (metadata2['miR_loc']=='UTR'), 'color'] = metadata2.loc[(metadata2['group']=='miR') & (metadata2['miR_loc']=='UTR'), 'color'].apply(base.get_light_color)

metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])), 'color'] = base.colors['blue']
metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts']=='FF6x1'), 'color'] = metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts']=='FF6x1'), 'color'].apply(base.get_dark_color)
metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts']=='FF4x1'), 'color'] = metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts']=='FF4x1'), 'color'].apply(base.get_light_color)

metadata2.loc[(metadata2['group']=='controller') & (metadata2['design']==1), 'color'] = base.colors['teal']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['design']==2), 'color'] = base.colors['orange']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['design']==3), 'color'] = base.colors['red']

metadata2.loc[(metadata2['group']=='controller') & (metadata2['ts_kind']=='NT'), 'color'] = base.colors['gray']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['ts_kind']=='NT') & (metadata2['design']==1), 'color'] = metadata2.loc[(metadata2['group']=='controller') & (metadata2['ts_kind']=='NT') & (metadata2['design']==1), 'color'].apply(base.get_light_color)
metadata2.loc[(metadata2['group']=='controller') & (metadata2['ts_kind']=='NT') & (metadata2['design']==3), 'color'] = metadata2.loc[(metadata2['group']=='controller') & (metadata2['ts_kind']=='NT') & (metadata2['design']==3), 'color'].apply(base.get_dark_color)

# markers
metadata2['markers'] = 'o'
metadata2.loc[(metadata2['group']=='base'), 'markers'] = 'X'
metadata2.loc[(metadata2['group']=='ts3'), 'markers'] = '^'
metadata2.loc[(metadata2['group']=='ts5'), 'markers'] = 'v'
metadata2.loc[(metadata2['miR'].isin(['miR.FF5','miR.FF4'])), 'markers'] = 'D'

# linestyles
metadata2['linestyle'] = '-' # default is solid
metadata2.loc[metadata2['group']=='marker', 'linestyle'] = ':'
metadata2.loc[(metadata2['group']=='ts5'), 'linestyle'] = '--'

metadata_dict2 = metadata2.set_index('construct').to_dict('dict')
designs_palette = metadata_dict2['color']
designs_markers = metadata_dict2['markers']

In [ ]:
# Create color palette for comparing promoters
metadata2 = metadata.copy()
metadata2.loc[(metadata2['group']=='marker'), 'color'] = 'black'
metadata2.loc[(metadata2['group']=='base'), 'color'] = 'black'

metadata2.loc[(metadata2['group']=='controller') & (metadata2['promoter']=='EF1a'), 'color'] = base.colors['teal']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['promoter']=='CAG'), 'color'] = base.colors['orange']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['promoter']=='EFS'), 'color'] = base.colors['red']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['promoter']=='hPGK'), 'color'] = base.colors['green']

metadata2.loc[(metadata2['ts_kind']=='NT'), 'color'] = base.colors['gray']
metadata2.loc[(metadata2['ts_kind']=='NT') & (metadata2['promoter']=='EF1a'), 'color'] = metadata2.loc[(metadata2['ts_kind']=='NT') & (metadata2['promoter']=='EF1a'), 'color'].apply(base.get_light_color)
metadata2.loc[(metadata2['ts_kind']=='NT') & (metadata2['promoter']=='EFS'), 'color'] = metadata2.loc[(metadata2['ts_kind']=='NT') & (metadata2['promoter']=='EFS'), 'color'].apply(base.get_dark_color)
metadata2.loc[(metadata2['ts_kind']=='NT') & (metadata2['promoter']=='hPGK'), 'color'] = metadata2.loc[(metadata2['ts_kind']=='NT') & (metadata2['promoter']=='hPGK'), 'color'].apply(base.get_dark_color).apply(base.get_dark_color)

# markers
metadata2['markers'] = 'o'
metadata2.loc[(metadata2['group']=='base'), 'markers'] = 'X'

# linestyles
metadata2['linestyle'] = '-' # default is solid
metadata2.loc[metadata2['group']=='marker', 'linestyle'] = ':'

metadata_dict2 = metadata2.set_index('construct').to_dict('dict')
promoter_palette = metadata_dict2['color']
promoter_markers = metadata_dict2['markers']

Draw gates based on untransfected population, then gate transfected cells

In [ ]:
gates = pd.DataFrame()
channel_list = ['tagBFP-A', 'mGL-A', 'mCherry-A', 'mRuby2-A']
for channel in channel_list:
    gates[channel] = data[data['construct']=='UT'].groupby(['exp'])[channel].apply(lambda x: x.quantile(0.999))
gates.reset_index(inplace=True)

# Add missing gates
gates.loc[len(gates.index)] = ['exp90.4',0,0,0,0,]  
gates.loc[gates['exp']=='exp90.4', channel_list] = gates.loc[gates['exp']=='exp90.2', channel_list].values

# Indicate which channels are relevant for each experiment
gates.sort_values(['exp'], inplace=True)
gates['marker'] = 'mGL-A'
gates['output'] = 'mRuby2-A'

display(gates)

Gate data per experiment based on transfection marker expression

In [ ]:
data = data.groupby('exp')[data.columns].apply(lambda x: base.gate_data(x,gates))
data.reset_index(inplace=True, drop=True)
max = 1e6
df = data[(data['expressing']) & (data['output']<max)]
display(df)

In [ ]:
# Bin by marker quantiles
by = ['construct','exp','biorep']
df['bin_marker_quantiles'] = df.groupby(by)['marker'].transform(lambda x: pd.qcut(x, q=20, duplicates='drop'))
quantiles = df.groupby(by+['bin_marker_quantiles'])['marker'].median().rename('bin_marker_quantiles_median').reset_index()
df_quantiles = df.merge(quantiles, how='left', on=by+['bin_marker_quantiles'])
display(df_quantiles)

In [ ]:
# Population stats
stat_list = [np.std, sp.stats.gmean, sp.stats.gstd, sp.stats.variation]
grouped = df_quantiles.groupby(by=['construct','biorep','exp'])
stats = grouped[['marker','output']].agg(stat_list).reset_index().dropna()

# Rename columns as 'col_stat'
stats.columns = stats.columns.map(lambda i: base.rename_multilevel_cols(i))
stats['count'] = grouped['output'].count().reset_index()['output']
stats = stats.merge(metadata, how='left', on='construct')
display(stats)

In [ ]:
# Quantile stats & slope
df_quantiles['bin_marker_quantiles_median_log'] = df_quantiles['bin_marker_quantiles_median'].apply(np.log10)
df_quantiles['output_log'] = df_quantiles['output'].apply(np.log10)

stat_list = [sp.stats.gmean, ]#sp.stats.gstd, sp.stats.variation]
grouped = df_quantiles.groupby(by=['construct','biorep','exp','bin_marker_quantiles_median'])
stats_quantiles = grouped[['marker','output']].agg(stat_list).reset_index().dropna()

# Rename columns as 'col_stat'
stats_quantiles.columns = stats_quantiles.columns.map(lambda i: base.rename_multilevel_cols(i))
stats_quantiles['count'] = grouped['output'].count().reset_index()['output']
stats_quantiles = stats_quantiles.merge(metadata, how='left', on='construct')
display(stats_quantiles)

stats_quantiles['bin_marker_quantiles_median_log'] = stats_quantiles['bin_marker_quantiles_median'].apply(np.log10)
stats_quantiles['output_gmean_log'] = stats_quantiles['output_gmean'].apply(np.log10)

In [ ]:
# Compute slope for all constructs
def get_slope(df):
    slope, intercept, r_value, p_value, stderr = sp.stats.linregress(df['bin_marker_quantiles_median_log'], df['output_gmean_log'])
    result = pd.DataFrame(columns=['slope', 'intercept_log', 'r_value', 'p_value', 'stderr'])
    result.loc[len(result.index)] = [slope, intercept, r_value, p_value, stderr]
    return result

fits = stats_quantiles.groupby(['construct','exp','biorep'])[stats_quantiles.columns].apply(get_slope).reset_index()
fits['intercept'] = fits['intercept_log'].apply(lambda x: 10**x)
fits = fits.merge(metadata, how='left', on='construct')
display(fits)

### Distribution plots

In [ ]:
# EF1a miR-FF5 design 1
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[(((df_quantiles['miR']=='miR.FF5') & (df_quantiles['group']=='controller') & (df_quantiles['design']==1) & (df_quantiles['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (df_quantiles['group']=='base')) &
                         (df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==biorep) & (df_quantiles['exp']!='exp90.4')]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_EF1a-miR.FF5.svg')), bbox_inches='tight')

In [ ]:
# EF1a miR-FF5 / miRE-FF5 design 1
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[(((df_quantiles['miR'].isin(['miR.FF5','miRE.FF5'])) & (df_quantiles['group']=='controller') & (df_quantiles['design']==1) & (df_quantiles['ts'].isin(['FF4x1','FF5x1','FF6x1'])))) &
                         (df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df[plot_df['miR'].isin(['miR.FF4','miR.FF5'])], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls='--')
sns.kdeplot(data=plot_df[plot_df['miR'].isin(['miRE.FF4','miRE.FF5'])], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls='-')
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_EF1a-miRE.FF5.svg')), bbox_inches='tight')

In [ ]:
# EF1a miR-FF4 / miRE-FF4 design 1
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[(((df_quantiles['miR'].isin(['miR.FF4','miRE.FF4'])) & (df_quantiles['group']=='controller') & (df_quantiles['design']==1) & (df_quantiles['ts'].isin(['FF4x1','FF5x1','FF6x1'])))) &
                         (df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df[plot_df['miR'].isin(['miR.FF4','miR.FF5'])], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls='--')
sns.kdeplot(data=plot_df[plot_df['miR'].isin(['miRE.FF4','miRE.FF5'])], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls='-')
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_EF1a-miRE.FF4.svg')), bbox_inches='tight')

In [ ]:
# EF1a miRE-FF5 / miRE-FF4 design 1
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[(((df_quantiles['miR'].isin(['miRE.FF5','miRE.FF4'])) & (df_quantiles['group']=='controller') & (df_quantiles['design']==1) & (df_quantiles['ts'].isin(['FF4x1','FF5x1','FF6x1'])))) &
                         (df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_EF1a-miRE.svg')), bbox_inches='tight')

In [ ]:
# EF1a miRE-FF4 designs 2,3
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[(((df_quantiles['miR']=='miRE.FF4') & (df_quantiles['group']=='controller') & (df_quantiles['design']>1) & (df_quantiles['ts'].isin(['FF4x1','FF5x1','FF6x1'])))) &
                         (df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
             legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_EF1a-miRE.FF4_designs2-3.svg')), bbox_inches='tight')

In [ ]:
# EF1a miRE-FF5 designs 2,3
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[(((df_quantiles['miR']=='miRE.FF5') & (df_quantiles['group']=='controller') & (df_quantiles['design']>1) & (df_quantiles['ts'].isin(['FF4x1','FF5x1','FF6x1'])))) &
                         (df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
             legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_EF1a-miRE.FF5_designs2-3.svg')), bbox_inches='tight')

In [ ]:
# EF1a miR-FF4 designs 2,3
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[(((df_quantiles['miR']=='miR.FF4') & (df_quantiles['group']=='controller') & (df_quantiles['design']>1) & (df_quantiles['ts'].isin(['FF4x1','FF5x1','FF6x1'])))) &
                         (df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
             legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_EF1a-miR.FF4_designs2-3.svg')), bbox_inches='tight')

In [ ]:
# EF1a miR-FF5 designs 2,3
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[(((df_quantiles['miR']=='miR.FF5') & (df_quantiles['group']=='controller') & (df_quantiles['design']>1) & (df_quantiles['ts'].isin(['FF4x1','FF5x1','FF6x1'])))) &
                         (df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
             legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_EF1a-miR.FF5_designs2-3.svg')), bbox_inches='tight')

In [ ]:
# EF1a miR-only
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[(((df_quantiles['group']=='miR')) & (df_quantiles['miR_loc']=='CDS')) &
                         (df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df[plot_df['miR'].isin(['miR.FF4','miR.FF5'])], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls='--')
sns.kdeplot(data=plot_df[plot_df['miR'].isin(['miRE.FF4','miRE.FF5'])], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls='-')
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_EF1a-miR-CDS.svg')), bbox_inches='tight')

In [ ]:
# EF1a miR-only
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[(((df_quantiles['group']=='miR')) & (df_quantiles['miR_loc']=='UTR')) &
                         (df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df[plot_df['miR'].isin(['miR.FF4','miR.FF5'])], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls='--')
sns.kdeplot(data=plot_df[plot_df['miR'].isin(['miRE.FF4','miRE.FF5'])], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls='-')
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_EF1a-miR-UTR.svg')), bbox_inches='tight')

In [ ]:
# EF1a ts-only
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[(((df_quantiles['group']=='ts3') & (df_quantiles['ts_num']==1))) &
                         (df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax,)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_EF1a-ts3.svg')), bbox_inches='tight')

In [ ]:
# EF1a ts-only
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[(((df_quantiles['group']=='ts5') & (df_quantiles['ts_num']==1))) &
                         (df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax,)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_EF1a-ts5.svg')), bbox_inches='tight')

In [ ]:
# hPGK miR-FF5 design 1
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[(((df_quantiles['miR']=='miR.FF5') & (df_quantiles['group']=='controller') & (df_quantiles['design']==1) & (df_quantiles['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (df_quantiles['group']=='base')) &
                         (df_quantiles['promoter']=='hPGK') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_hPGK-miR.FF5.svg')), bbox_inches='tight')

In [ ]:
# hPGK miRE-FF5 / miRE-FF4 design 1
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[(((df_quantiles['miR'].isin(['miRE.FF5','miRE.FF4'])) & (df_quantiles['group']=='controller') & (df_quantiles['design']==1) & (df_quantiles['ts'].isin(['FF4x1','FF5x1','FF6x1'])))) &
                         (df_quantiles['promoter']=='hPGK') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_hPGK-miRE.svg')), bbox_inches='tight')

In [ ]:
# hPGK miRE-FF4 designs 2,3
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[(((df_quantiles['miR']=='miRE.FF4') & (df_quantiles['group']=='controller') & (df_quantiles['design']>1) & (df_quantiles['ts'].isin(['FF4x1','FF5x1','FF6x1'])))) &
                         (df_quantiles['promoter']=='hPGK') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
             legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_hPGK-miRE.FF4_designs2-3.svg')), bbox_inches='tight')

In [ ]:
# hPGK miR-only
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[(((df_quantiles['group']=='miR')) & (df_quantiles['miR_loc']=='CDS')) &
                         (df_quantiles['promoter']=='hPGK') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df[plot_df['miR'].isin(['miR.FF4','miR.FF5'])], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls='--')
sns.kdeplot(data=plot_df[plot_df['miR'].isin(['miRE.FF4','miRE.FF5'])], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls='-')
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_hPGK-miR-CDS.svg')), bbox_inches='tight')

In [ ]:
# hPGK miR-only
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[(((df_quantiles['group']=='miR')) & (df_quantiles['miR_loc']=='UTR')) &
                         (df_quantiles['promoter']=='hPGK') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df[plot_df['miR'].isin(['miR.FF4','miR.FF5'])], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls='--')
sns.kdeplot(data=plot_df[plot_df['miR'].isin(['miRE.FF4','miRE.FF5'])], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls='-')
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_hPGK-miR-UTR.svg')), bbox_inches='tight')

In [ ]:
# hPGK ts-only
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[(((df_quantiles['group']=='ts3') & (df_quantiles['ts_num']==1))) &
                         (df_quantiles['promoter']=='hPGK') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax,)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_hPGK-ts3.svg')), bbox_inches='tight')

In [ ]:
# hPGK ts-only
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[(((df_quantiles['group']=='ts5') & (df_quantiles['ts_num']==1))) &
                         (df_quantiles['promoter']=='hPGK') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax,)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_hPGK-ts5.svg')), bbox_inches='tight')

In [ ]:
# EF1a miRE-FF4 design 1, vary num ts
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 1
plot_df = df_quantiles[(((df_quantiles['miR']=='miRE.FF4') & (df_quantiles['group']=='controller') & (df_quantiles['design']==1))) &
                         (df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_EF1a-miRE.FF4-num-ts.svg')), bbox_inches='tight')

In [ ]:
# miRE-FF4 design 1, vary promoters
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[(((df_quantiles['miR']=='miRE.FF4') & (df_quantiles['group']=='controller') & (df_quantiles['design']==1) & (df_quantiles['ts_num']==1)) | (df_quantiles['group']=='base')) &
                         (df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_EF1a-miRE.FF4.svg')), bbox_inches='tight')

In [ ]:
# miRE-FF4 design 1, vary promoters
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[(((df_quantiles['miR']=='miRE.FF4') & (df_quantiles['group']=='controller') & (df_quantiles['design']==1) & (df_quantiles['ts_num']==1)) | (df_quantiles['group']=='base')) &
                         (df_quantiles['promoter']=='CAG') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_CAG-miRE.FF4.svg')), bbox_inches='tight')

In [ ]:
# miRE-FF4 design 1, vary promoters
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 1
plot_df = df_quantiles[(((df_quantiles['miR']=='miRE.FF4') & (df_quantiles['group']=='controller') & (df_quantiles['design']==1) & (df_quantiles['ts_num']==1)) | (df_quantiles['group']=='base')) &
                         (df_quantiles['promoter']=='EFS') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_EFS-miRE.FF4.svg')), bbox_inches='tight')

In [ ]:
# miRE-FF4 design 1, vary promoters
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[(((df_quantiles['miR']=='miRE.FF4') & (df_quantiles['group']=='controller') & (df_quantiles['design']==1) & (df_quantiles['ts_num']==1)) | (df_quantiles['group']=='base')) &
                         (df_quantiles['promoter']=='hPGK') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_hPGK-miRE.FF4.svg')), bbox_inches='tight')

In [ ]:
# miRE-FF4 design 1, FSC
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (1e4,1e5) #(2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[(((df_quantiles['miR']=='miRE.FF4') & (df_quantiles['group']=='controller') & (df_quantiles['design']==1) & (df_quantiles['ts_num']==1)) | (df_quantiles['group']=='base')) &
                         (df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.kdeplot(data=plot_df, x='FSC-A', y='output_log', hue='construct', palette=main_palette, 
            legend=False, common_norm=False, fill=False, levels=6)
ax.set(xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
#sns.kdeplot(data=plot_df, y='output', hue='construct', palette=main_palette, 
#             legend=False, log_scale=True, common_norm=False, ax=ax)
#sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=main_palette, 
#             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'kde-marg-hist-FSC_EF1a-miRE.FF4.svg')), bbox_inches='tight')

### Stat plots

In [ ]:
# EF1a miR-FF5 / miRE-FF5 / miRE-FF4 design 1
fig, axes = plt.subplots(1,3, figsize=(10,5), gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.4))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
miR_order = ['none', 'miR.FF5', 'miRE.FF5', 'miRE.FF4', 'miR.FF4',]
plot_df = stats[(((stats['miR'].isin(miR_order[1:])) & (stats['group']=='controller') & (stats['design']==1) & (stats['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (stats['group']=='base')) &
                (stats['promoter']=='EF1a')].copy()
plot_df['miR'] = plot_df['miR'].astype(pd.api.types.CategoricalDtype(categories=miR_order, ordered=True))
plot_df2 = fits[(((fits['miR'].isin(miR_order[1:])) & (fits['group']=='controller') & (fits['design']==1) & (fits['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (fits['group']=='base')) &
                (fits['promoter']=='EF1a')].copy()
plot_df2['miR'] = plot_df2['miR'].astype(pd.api.types.CategoricalDtype(categories=miR_order, ordered=True))

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='miR', y='output_gmean', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(miR_order)-1.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='miR', y='output_std', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(miR_order)-1.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct'):
    sns.stripplot(data=group, x='miR', y='slope', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(miR_order)-1.5), xlabel='', ylim=(0,1.2), ylabel='',)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

fig.savefig(rd.outfile(output_path/(f'stat_EF1a-miR-main.svg')), bbox_inches='tight')

In [ ]:
# EF1a all miR design 1
fig, axes = plt.subplots(1,3, figsize=(10,5), gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.4))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
miR_order = ['none', 'miR.FF5', 'miRE.FF5', 'miRE.FF4', 'miR.FF4',]
plot_df = stats[(((stats['miR'].isin(miR_order[1:])) & (stats['group']=='controller') & (stats['design']==1) & (stats['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (stats['group']=='base')) &
                (stats['promoter']=='EF1a')].copy()
plot_df['miR'] = plot_df['miR'].astype(pd.api.types.CategoricalDtype(categories=miR_order, ordered=True))
plot_df2 = fits[(((fits['miR'].isin(miR_order[1:])) & (fits['group']=='controller') & (fits['design']==1) & (fits['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (fits['group']=='base')) &
                (fits['promoter']=='EF1a')].copy()
plot_df2['miR'] = plot_df2['miR'].astype(pd.api.types.CategoricalDtype(categories=miR_order, ordered=True))

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='miR', y='output_gmean', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(miR_order)-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='miR', y='output_std', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(miR_order)-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct'):
    sns.stripplot(data=group, x='miR', y='slope', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(miR_order)-0.5), xlabel='', ylim=(0,1.2), ylabel='',)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

fig.savefig(rd.outfile(output_path/(f'stat_EF1a-miR-all.svg')), bbox_inches='tight')

In [ ]:
# EF1a miRE-FF4 all designs
fig, axes = plt.subplots(1,3, figsize=(10,5), gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.4))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
plot_df = stats[(((stats['miR']=='miRE.FF4') & (stats['group']=='controller') & (stats['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (stats['group']=='base')) &
                (stats['promoter']=='EF1a')].copy()
plot_df.sort_values('design', inplace=True)
plot_df2 = fits[(((fits['miR']=='miRE.FF4') & (fits['group']=='controller') & (fits['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (fits['group']=='base')) &
                (fits['promoter']=='EF1a')].copy()
plot_df2.sort_values('design', inplace=True)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='design', y='output_gmean', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(plot_df['design'].unique())-0.5), ylim=ylim, ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='design', y='output_std', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(plot_df['design'].unique())-0.5), ylim=ylim, ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct'):
    sns.stripplot(data=group, x='design', y='slope', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(plot_df2['design'].unique())-0.5), ylim=(0,1.2), ylabel='',)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

fig.savefig(rd.outfile(output_path/(f'stat_EF1a-miRE.FF4-designs.svg')), bbox_inches='tight')

In [ ]:
# EF1a miRE-FF5 all designs
fig, axes = plt.subplots(1,3, figsize=(10,5), gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.4))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
plot_df = stats[(((stats['miR']=='miRE.FF5') & (stats['group']=='controller') & (stats['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (stats['group']=='base')) &
                (stats['promoter']=='EF1a')].copy()
plot_df.sort_values('design', inplace=True)
plot_df2 = fits[(((fits['miR']=='miRE.FF5') & (fits['group']=='controller') & (fits['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (fits['group']=='base')) &
                (fits['promoter']=='EF1a')].copy()
plot_df2.sort_values('design', inplace=True)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='design', y='output_gmean', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(plot_df['design'].unique())-0.5), ylim=ylim, ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='design', y='output_std', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(plot_df['design'].unique())-0.5), ylim=ylim, ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct'):
    sns.stripplot(data=group, x='design', y='slope', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(plot_df2['design'].unique())-0.5), ylim=(0,1.2), ylabel='',)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

fig.savefig(rd.outfile(output_path/(f'stat_EF1a-miRE.FF5-designs.svg')), bbox_inches='tight')

In [ ]:
# EF1a miR-FF5 all designs
fig, axes = plt.subplots(1,3, figsize=(10,5), gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.4))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
plot_df = stats[(((stats['miR']=='miR.FF5') & (stats['group']=='controller') & (stats['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (stats['group']=='base')) &
                (stats['promoter']=='EF1a')].copy()
plot_df.sort_values('design', inplace=True)
plot_df2 = fits[(((fits['miR']=='miR.FF5') & (fits['group']=='controller') & (fits['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (fits['group']=='base')) &
                (fits['promoter']=='EF1a')].copy()
plot_df2.sort_values('design', inplace=True)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='design', y='output_gmean', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(plot_df['design'].unique())-0.5), ylim=ylim, ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='design', y='output_std', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(plot_df['design'].unique())-0.5), ylim=ylim, ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct'):
    sns.stripplot(data=group, x='design', y='slope', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(plot_df2['design'].unique())-0.5), ylim=(0,1.2), ylabel='',)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

fig.savefig(rd.outfile(output_path/(f'stat_EF1a-miR.FF5-designs.svg')), bbox_inches='tight')

In [ ]:
# EF1a miR-FF4 all designs
fig, axes = plt.subplots(1,3, figsize=(10,5), gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.4))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
plot_df = stats[(((stats['miR']=='miR.FF4') & (stats['group']=='controller') & (stats['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (stats['group']=='base')) &
                (stats['promoter']=='EF1a')].copy()
plot_df.sort_values('design', inplace=True)
plot_df2 = fits[(((fits['miR']=='miR.FF4') & (fits['group']=='controller') & (fits['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (fits['group']=='base')) &
                (fits['promoter']=='EF1a')].copy()
plot_df2.sort_values('design', inplace=True)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='design', y='output_gmean', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(plot_df['design'].unique())-0.5), ylim=ylim, ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='design', y='output_std', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(plot_df['design'].unique())-0.5), ylim=ylim, ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct'):
    sns.stripplot(data=group, x='design', y='slope', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(plot_df2['design'].unique())-0.5), ylim=(0,1.2), ylabel='',)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

fig.savefig(rd.outfile(output_path/(f'stat_EF1a-miR.FF4-designs.svg')), bbox_inches='tight')

In [ ]:
# EF1a miR controls design 1
fig, axes = plt.subplots(1,3, figsize=(10,5), gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.4))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
miR_order = ['none', 'miR.FF5', 'miRE.FF5', 'miRE.FF4', 'miR.FF4',]
plot_df = stats[((((stats['group']=='miR')) & (stats['miR_loc']=='CDS')) | (stats['group']=='base')) &
                (stats['promoter']=='EF1a')].copy()
plot_df['miR'] = plot_df['miR'].astype(pd.api.types.CategoricalDtype(categories=miR_order, ordered=True))
plot_df2 = fits[((((fits['group']=='miR')) & (fits['miR_loc']=='CDS')) | (fits['group']=='base')) &
                (fits['promoter']=='EF1a')].copy()
plot_df2['miR'] = plot_df2['miR'].astype(pd.api.types.CategoricalDtype(categories=miR_order, ordered=True))

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='miR', y='output_gmean', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='miR', y='output_std', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct'):
    sns.stripplot(data=group, x='miR', y='slope', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(plot_df2['construct'].unique())-0.5), xlabel='', ylim=(0,1.2), ylabel='',)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

fig.savefig(rd.outfile(output_path/(f'stat_EF1a-miR-controls-CDS.svg')), bbox_inches='tight')

In [ ]:
# EF1a miR controls designs 2,3
fig, axes = plt.subplots(1,3, figsize=(10,5), gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.4))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
miR_order = ['none', 'miR.FF5', 'miRE.FF5', 'miRE.FF4', 'miR.FF4',]
plot_df = stats[((((stats['group']=='miR')) & (stats['miR_loc']=='UTR')) | (stats['group']=='base')) &
                (stats['promoter']=='EF1a')].copy()
plot_df['miR'] = plot_df['miR'].astype(pd.api.types.CategoricalDtype(categories=miR_order, ordered=True))
plot_df2 = fits[((((fits['group']=='miR')) & (fits['miR_loc']=='UTR')) | (fits['group']=='base')) &
                (fits['promoter']=='EF1a')].copy()
plot_df2['miR'] = plot_df2['miR'].astype(pd.api.types.CategoricalDtype(categories=miR_order, ordered=True))

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='miR', y='output_gmean', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='miR', y='output_std', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct'):
    sns.stripplot(data=group, x='miR', y='slope', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(plot_df2['construct'].unique())-0.5), xlabel='', ylim=(0,1.2), ylabel='',)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

fig.savefig(rd.outfile(output_path/(f'stat_EF1a-miR-controls-UTR.svg')), bbox_inches='tight')

In [ ]:
# EF1a ts controls 3'UTR
fig, axes = plt.subplots(1,3, figsize=(10,5), gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.4))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
ts_order = ['none','FF6x1','FF5x1','FF4x1','FF3x1']
plot_df = stats[((((stats['group']=='ts3')) & (stats['ts_num']==1)) | (stats['group']=='base')) &
                (stats['promoter']=='EF1a')].copy()
plot_df['ts'] = plot_df['ts'].astype(pd.api.types.CategoricalDtype(categories=ts_order, ordered=True))
plot_df2 = fits[((((fits['group']=='ts3')) & (fits['ts_num']==1)) | (fits['group']=='base')) &
                (fits['promoter']=='EF1a')].copy()
plot_df2['ts'] = plot_df2['ts'].astype(pd.api.types.CategoricalDtype(categories=ts_order, ordered=True))

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts', y='output_gmean', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(plot_df['ts'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts', y='output_std', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(plot_df['ts'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct'):
    sns.stripplot(data=group, x='ts', y='slope', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(plot_df2['ts'].unique())-0.5), xlabel='', ylim=(0,1.2), ylabel='',)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

fig.savefig(rd.outfile(output_path/(f'stat_EF1a-ts3-controls.svg')), bbox_inches='tight')

In [ ]:
# EF1a ts controls 5'UTR
fig, axes = plt.subplots(1,3, figsize=(10,5), gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.4))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
ts_order = ['none','FF6x1','FF5x1','FF4x1','FF3x1']
plot_df = stats[((((stats['group']=='ts5')) & (stats['ts_num']==1)) | (stats['group']=='base')) &
                (stats['promoter']=='EF1a')].copy()
plot_df['ts'] = plot_df['ts'].astype(pd.api.types.CategoricalDtype(categories=ts_order, ordered=True))
plot_df2 = fits[((((fits['group']=='ts5')) & (fits['ts_num']==1)) | (fits['group']=='base')) &
                (fits['promoter']=='EF1a')].copy()
plot_df2['ts'] = plot_df2['ts'].astype(pd.api.types.CategoricalDtype(categories=ts_order, ordered=True))

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts', y='output_gmean', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(plot_df['ts'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts', y='output_std', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(plot_df['ts'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct'):
    sns.stripplot(data=group, x='ts', y='slope', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(plot_df2['ts'].unique())-0.5), xlabel='', ylim=(0,1.2), ylabel='',)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

fig.savefig(rd.outfile(output_path/(f'stat_EF1a-ts5-controls.svg')), bbox_inches='tight')

In [ ]:
# EF1a miRE.FF4 design 1 ts num
fig, axes = plt.subplots(1,3, figsize=(10,5), gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.4))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
ts_order = ['none','FF6x1','FF4x1','FF6x2','FF4x2','FF6x4','FF4x4']
plot_df = stats[((((stats['miR']=='miRE.FF4')) & (stats['group']=='controller') & (stats['design']==1)) | (stats['group']=='base')) &
                (stats['promoter']=='EF1a')].copy()
plot_df['ts'] = plot_df['ts'].astype(pd.api.types.CategoricalDtype(categories=ts_order, ordered=True))
plot_df2 = fits[((((fits['miR']=='miRE.FF4')) & (fits['group']=='controller') & (fits['design']==1)) | (fits['group']=='base')) &
                (fits['promoter']=='EF1a')].copy()
plot_df2['ts'] = plot_df2['ts'].astype(pd.api.types.CategoricalDtype(categories=ts_order, ordered=True))

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts', y='output_gmean', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(plot_df['ts'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts', y='output_std', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(plot_df['ts'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct'):
    sns.stripplot(data=group, x='ts', y='slope', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(plot_df2['ts'].unique())-0.5), xlabel='', ylim=(0,1.2), ylabel='',)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

fig.savefig(rd.outfile(output_path/(f'stat_EF1a-miRE.FF4-ts-num.svg')), bbox_inches='tight')

In [ ]:
# EF1a ts num controls
fig, axes = plt.subplots(1,3, figsize=(10,5), gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.4))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
ts_order = ['none','FF6x1','FF4x1','FF6x2','FF4x2','FF6x4','FF4x4']
plot_df = stats[((stats['group']=='ts3') | (stats['group']=='base')) &
                (stats['promoter']=='EF1a')].copy()
plot_df['ts'] = plot_df['ts'].astype(pd.api.types.CategoricalDtype(categories=ts_order, ordered=True))
plot_df2 = fits[((fits['group']=='ts3') | (fits['group']=='base')) &
                (fits['promoter']=='EF1a')].copy()
plot_df2['ts'] = plot_df2['ts'].astype(pd.api.types.CategoricalDtype(categories=ts_order, ordered=True))

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts', y='output_gmean', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(plot_df['ts'].unique())-1.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts', y='output_std', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(plot_df['ts'].unique())-1.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct'):
    sns.stripplot(data=group, x='ts', y='slope', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(plot_df2['ts'].unique())-1.5), xlabel='', ylim=(0,1.2), ylabel='',)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

fig.savefig(rd.outfile(output_path/(f'stat_EF1a-controls-ts-num.svg')), bbox_inches='tight')

In [ ]:
# miR.FF4 promoters
fig, axes = plt.subplots(1,3, figsize=(18,5), gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.15))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
promoter_order = ['EF1a','CAG','EFS','hPGK']
plot_df = stats[(((stats['miR']=='miRE.FF4') & (stats['group']=='controller') & (stats['design']==1) & (stats['ts_num']==1)) | (stats['group']=='base'))].copy()
plot_df['promoter'] = plot_df['promoter'].astype(pd.api.types.CategoricalDtype(categories=promoter_order, ordered=True))
plot_df.sort_values(['promoter','group','ts_kind'], inplace=True)
plot_df2 = fits[(((fits['miR']=='miRE.FF4') & (fits['group']=='controller') & (fits['design']==1) & (fits['ts_num']==1)) | (fits['group']=='base'))].copy()
plot_df2['promoter'] = plot_df2['promoter'].astype(pd.api.types.CategoricalDtype(categories=promoter_order, ordered=True))
plot_df2.sort_values(['promoter','group','ts_kind'], inplace=True)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct', y='output_gmean', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
rd.plot.generate_xticklabels(metadata2, 'construct', ['ts_kind'], annotate=False, ax=ax)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct', y='output_std', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
rd.plot.generate_xticklabels(metadata2, 'construct', ['ts_kind'], annotate=False, ax=ax)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct', y='slope', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(plot_df2['construct'].unique())-0.5), xlabel='', ylim=(0,1.2), ylabel='',)
rd.plot.generate_xticklabels(metadata2, 'construct', ['ts_kind'], annotate=False, ax=ax)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

for i,ax in enumerate(axes):
    ax.axvspan(2.5, 5.5, color=base.get_light_color(base.colors['gray']), alpha=0.2,)
    ax.axvspan(9.5, 13.5, color=base.get_light_color(base.colors['gray']), alpha=0.2,)
    ax.annotate('EF1a', (1,ax.get_ylim()[1]), ha='center', va='top', fontsize=sns.plotting_context('talk')['legend.fontsize'])
    ax.annotate('CAG', (4,ax.get_ylim()[1]), ha='center', va='top', fontsize=sns.plotting_context('talk')['legend.fontsize'])
    ax.annotate('EFS', (7.5,ax.get_ylim()[1]), ha='center', va='top', fontsize=sns.plotting_context('talk')['legend.fontsize'])
    ax.annotate('hPGK', (11.5,ax.get_ylim()[1]), ha='center', va='top', fontsize=sns.plotting_context('talk')['legend.fontsize'])

fig.savefig(rd.outfile(output_path/(f'stat_miRE.FF4-promoters.svg')), bbox_inches='tight')

In [ ]:
# miR.FF4 promoters
fig, axes = plt.subplots(1,3, figsize=(18,5), gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.15))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
promoter_order = ['EF1a','CAG','EFS','hPGK']
plot_df = stats[(((stats['miR']=='miRE.FF4') & (stats['group']=='controller') & (stats['design']==1) & (stats['ts'].isin(['FF4x1','FF6x1']))) | (stats['group']=='base'))].copy()
plot_df['promoter'] = plot_df['promoter'].astype(pd.api.types.CategoricalDtype(categories=promoter_order, ordered=True))
plot_df.sort_values(['promoter','group','ts_kind'], inplace=True)
plot_df2 = fits[(((fits['miR']=='miRE.FF4') & (fits['group']=='controller') & (fits['design']==1) & (fits['ts'].isin(['FF4x1','FF6x1']))) | (fits['group']=='base'))].copy()
plot_df2['promoter'] = plot_df2['promoter'].astype(pd.api.types.CategoricalDtype(categories=promoter_order, ordered=True))
plot_df2.sort_values(['promoter','group','ts_kind'], inplace=True)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct', y='output_gmean', hue='construct', palette=promoter_palette,
                    legend=False, ax=ax, marker=promoter_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
rd.plot.generate_xticklabels(metadata2, 'construct', ['ts_kind'], annotate=False, ax=ax)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct', y='output_std', hue='construct', palette=promoter_palette,
                    legend=False, ax=ax, marker=promoter_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
rd.plot.generate_xticklabels(metadata2, 'construct', ['ts_kind'], annotate=False, ax=ax)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct', y='slope', hue='construct', palette=promoter_palette,
                    legend=False, ax=ax, marker=promoter_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(plot_df2['construct'].unique())-0.5), xlabel='', ylim=(0,1.2), ylabel='',)
rd.plot.generate_xticklabels(metadata2, 'construct', ['ts_kind'], annotate=False, ax=ax)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

for i,ax in enumerate(axes):
    ax.axvspan(2.5, 5.5, color=base.get_light_color(base.colors['gray']), alpha=0.2,)
    ax.axvspan(8.5, 11.5, color=base.get_light_color(base.colors['gray']), alpha=0.2,)
    ax.annotate('EF1a', (1,ax.get_ylim()[1]), ha='center', va='top', fontsize=sns.plotting_context('talk')['legend.fontsize'])
    ax.annotate('CAG', (4,ax.get_ylim()[1]), ha='center', va='top', fontsize=sns.plotting_context('talk')['legend.fontsize'])
    ax.annotate('EFS', (7,ax.get_ylim()[1]), ha='center', va='top', fontsize=sns.plotting_context('talk')['legend.fontsize'])
    ax.annotate('hPGK', (10,ax.get_ylim()[1]), ha='center', va='top', fontsize=sns.plotting_context('talk')['legend.fontsize'])

fig.savefig(rd.outfile(output_path/(f'stat_miRE.FF4-promoters_alt.svg')), bbox_inches='tight')